# 10. ACP

Dans ce notebook, nous allons appliquer une ACP sur différents datasets en utilisant la librairie [scikit-learn](https://scikit-learn.org/stable/).

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, normalize, StandardScaler

## 10.1 ACP pour la compréhension et l'analyse d'un dataset

Nous allons appliquer une ACP sur un jeu de données décrivant quelques statistiques sur des pays. L'objectif est d'identifier de trouver une catégorisation des pays en fonction de ces donnés.

Charger le dataset contenu dans le fichier `data/country-data.csv` dans une dataframe pandas, visualisez un échantillon de ces données, afficher quelques statistiques (mean, std...) et donnez sa taille.

In [ ]:
df = pd.read_csv('/data/country-data.csv')
df.head()

In [ ]:
df.shape

Nous nous intéressons à présent aux corrélations linéaires entre les variables de ce dataset à l'aide de la méthode [`pandas.DataFrame.corr`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html).

Afficher la matrice des corrélations avec la méthode [`sns.heatmap`](https://seaborn.pydata.org/generated/seaborn.heatmap.html). Vous pouvez utiliser le code `plt.figure(figsize=(10,10))` au préalable pour obtenir une figure plus lisible. Pour afficher cette matrice, les paramètres intéressants de `sns.heatmap` sont `square`, `annot` et `cmap` (utilisez l'espace de couleur `sns.diverging_palette(20, 220, n=200)` pour plus de lisibilité).

In [ ]:
correlation = df.corr()
plt.figure(figsize=(10,10))
sns.heatmap(correlation, vmax=1, square=True,annot=True, cmap=sns.diverging_palette(20, 220, n=200))

plt.title('Correlation between different fearures')

Afficher les individus du dataset pour quelques paires de variables fortement corrélées. Vous pouvez utiliser un [scatter plot plotly](https://plotly.com/python/line-and-scatter/). 

Quelles conclusions pouvezèvous tirer des variables et des individus ?

In [ ]:
fig = px.scatter(df, x="child_mort", y="life_expec", text="country")
fig.show()

Créez une variable `X` contenant les données numériques des individus (i.e. : sans la colonne `country`). Puis appliquez la méthode `fit_transform` de la classe [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) de scikit-lean, stockez le résultat dans une nouvelle variable `X_std`.

In [ ]:
X = df.drop(columns='country')

In [ ]:
X_std = StandardScaler().fit_transform(X)
# df_std = StandardScaler().fit_transform(df.drop(columns='country'))
X_std

Afficher la distribution des variables de ce dataset avant et après la standardisation en utilisant [`pandas.DataFrame.plot.density`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.density.html) (les paramètres `sharex=True,figsize=(12,5)` rendront ce graphique plus lisible).

Sur le graphique de la distribution des variables avant standardisation, certaines variables dont la plage des valeurs est trop différentes des autres rendrait ce graphique illisible. Pensez à afficher un graphique différent pour chaque groupe de variables ayant une plage de valeurs proche.

In [ ]:
df.drop(columns=['income', 'gdpp']).plot.density(sharex=True,figsize=(12,5))

In [ ]:
df[['income', 'gdpp']].plot.density(sharex=True,figsize=(12,5))

In [ ]:
features = df.drop(columns='country').columns.tolist()
features

In [ ]:
pd.DataFrame(X_std, columns=features).plot.density(sharex=True,figsize=(12,5))

Appliquez la méthode `fit` de la classe [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) pour calculer les composantes principales. Puis afficher les valeurs des attributs `components_`, `explained_variance_` et `explained_variance_ratio_`. A quoi correspondent ces attributs ?

In [ ]:
pca = PCA().fit(X_std)

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_

In [ ]:
pca.explained_variance_ratio_

L'attribut `components_` contient les coefficients des combinaisons linéaires des différentes composantes principales, l'attribut `explained_variance_` fournit la variance expliquée sur le dataset pour chacune des composantes principales et l'attribut `explained_variance_ratio_` la proportion de variance expliquée cumulée par les composantes principales.

La fonction suivante vous permet d'afficher un graphique représentant la variance expliquée par les différentes composantes principales et la variance cumulée. Utilisez cette fonction sur votre ACP. Combien faut-il conserver de cmposantes principales pour expliquer plus de 80% de la variabilité des données ?

In [ ]:
def display_scree_plot(pca):
    '''Display a scree plot for the pca'''

    scree = pca.explained_variance_ratio_*100
    plt.figure(figsize=(15,8))
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("Number of principal components")
    plt.ylabel("Percentage explained variance")
    plt.title("Scree plot")
    plt.show(block=False)

In [ ]:
display_scree_plot(pca)

Les quatre premières composantes principales suffisent à expliquer plus de 80% de la variance de nos données.

La fonction suivante vous permet d'afficher le cercle des corrélations des variables. Ses paramètres principaux sont :
* `pcs` : ndarray, les composantes principales
* `n_comp` : int, le nombre de composantes
* `pca` : sklearn.decomposition.PCA, l'ACP
* `axis_ranks` : list, les indices des paires d'axes à afficher (chaque paire affichera un nouveau cercle de corrélation), exemple : [(0,1)]
* `labels` : list, le nom des variables

Appliquez la pour visualiser la projection des variables dans le premier et le second plan factoriel. Interprétez les composantes principales affichées.

In [ ]:
def display_circles(pcs, n_comp, pca, axis_ranks, labels=None, label_rotation=0, lims=None):
    """Display correlation circles, one for each factorial plane"""

    # For each factorial plane
    for d1, d2 in axis_ranks: 
        if d2 < n_comp:

            # Initialise the matplotlib figure
            fig, ax = plt.subplots(figsize=(10,10))

            # Determine the limits of the chart
            if lims is not None :
                xmin, xmax, ymin, ymax = lims
            elif pcs.shape[1] < 30 :
                xmin, xmax, ymin, ymax = -1, 1, -1, 1
            else :
                xmin, xmax, ymin, ymax = min(pcs[d1,:]), max(pcs[d1,:]), min(pcs[d2,:]), max(pcs[d2,:])

            # Add arrows
            # If there are more than 30 arrows, we do not display the triangle at the end
            if pcs.shape[1] < 30 :
                plt.quiver(np.zeros(pcs.shape[1]), np.zeros(pcs.shape[1]),
                   pcs[d1,:], pcs[d2,:], 
                   angles='xy', scale_units='xy', scale=1, color="grey")
                # (see the doc : https://matplotlib.org/api/_as_gen/matplotlib.pyplot.quiver.html)
            else:
                lines = [[[0,0],[x,y]] for x,y in pcs[[d1,d2]].T]
                ax.add_collection(LineCollection(lines, axes=ax, alpha=.1, color='black'))
            
            # Display variable names
            if labels is not None:  
                for i,(x, y) in enumerate(pcs[[d1,d2]].T):
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax :
                        plt.text(x, y, labels[i], fontsize='14', ha='center', va='center', rotation=label_rotation, color="blue", alpha=0.5)
            
            # Display circle
            circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
            plt.gca().add_artist(circle)

            # Define the limits of the chart
            plt.xlim(xmin, xmax)
            plt.ylim(ymin, ymax)
        
            # Display grid lines
            plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # Label the axes, with the percentage of variance explained
            plt.xlabel('PC{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('PC{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Correlation Circle (PC{} and PC{})".format(d1+1, d2+1))
            plt.show(block=False)

In [ ]:
pcs = pca.components_
display_circles(pcs, 9, pca, [(0,1), (0, 2), (2, 3)], labels = features)

Sur le premier plan factoriel, nous observons que les variables `imports` et `exports` sont liées positivement à la deuxième composante principale. Dans une moindre mesure, les variables `income` et `gdpp` sont liées positivement à la première composante principale, et les variables `child_mort` et `total_fer` négativement.

Appliquez la fonction `transform` sur votre objet PCA pour appliquer l'ACP sur le dataset centré réduit et stockez le résultats dans une variable `X_pca`. Puis crééz une dataframe contenant le résultat de l'ACP.

In [ ]:
X_pca = pca.transform(X_std)
X_pca

In [ ]:
df_pca = pd.DataFrame(X_pca, index=df['country'])
df_pca

Afficher la projection des individus sur les deux premiers plans factoriels. Vous pouvez utiliser un [scatter plot plotly](https://plotly.com/python/line-and-scatter/) avec la dataframe du résultat de l'ACP (pour faciliter la lecture du graphique, vous pouvez utiliser les paramètres `hover_name` ou `text` en utilisant la colonne `country` de votre dataframe d'origine).

Liez ce que vous observez avec l'analyse du cercle des corrélation de ce plan factoriel. Vous pouvez faire de même pour d'autres plans factoriels.

In [ ]:
fig = px.scatter(df_pca, x=0, y=1, hover_name=df.country, text=df.country)
fig.show()

La première composante principale (en abscisse) distingue les pays riches des pays en voie de développement, ce qui s'explique par l'importance du niveau de vie et de la mortalité infantile observée sur le cercle de corrélation pour cette composante. La deuxième composante principale semble difficilement explicable. Il en est de même pour le deuxième plan factoriel :

In [ ]:
fig = px.scatter(df_pca, x=2, y=3, hover_name=df.country, text=df.country)
fig.show()

## 10.2 ACP sur des données synthétiques

Nous allons illustrer l'effet de la normalisation et de la mise à l'échelle sur l'ACP appliqué à un dataset synthétique.

Pour cela on se dote d'un dataset contenant 4 variables respectant une distribution normale et une 5ème valant 0 ou 3.

La cellule suivante créé un tel dataset en utilisant numpy:

In [ ]:
import numpy as np

np.random.seed(123)    # For reproducibility

N = 200
P = 5
rho = 0.5

X = np.random.normal(size=[N,P])
X = np.append(X, 3*np.random.choice(2, size = [N,1]), axis = 1)
X

Examiner votre dataset et ses propriétés en crééant une dataframe à partir de `X`.

In [ ]:
df_synthetic = pd.DataFrame(X)
df_synthetic.head()

In [ ]:
df_synthetic.describe()

Afficher la distribution des variables de ce dataset en utilisant [`pandas.DataFrame.plot.density`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.density.html) (les paramètres `sharex=True,figsize=(12,5)` rendront ce graphique plus lisible).

In [ ]:
df_synthetic.plot.density(sharex=True,figsize=(12,5),layout=(10,1))

Appliquer une ACP sans aucun pré-traitement. Que constatez-vous ?

In [ ]:
pca = PCA(2)
low_d = pca.fit_transform(X)
plt.scatter(low_d[:,0], low_d[:,1])

Appliquez une ACP après avoir réduit vos données ([`sklearn.preprocessing.normalize`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html)). Que constatez-vous ?

In [ ]:
# normalize
Xn = normalize(X)
pca = PCA(2)
low_d = pca.fit_transform(Xn)
plt.scatter(low_d[:,0], low_d[:,1])

Appliquez une ACP après avoir centré vos données ([`sklearn.preprocessing.scale`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html)). Que constatez-vous ?

In [ ]:
# Scale
Xs = scale(X)
low_d = pca.fit_transform(Xs)
plt.scatter(low_d[:,0], low_d[:,1])

In [ ]:
# Scale and normalize
Xs = StandardScaler().fit_transform(X)
low_d = pca.fit_transform(Xs)
plt.scatter(low_d[:,0], low_d[:,1])

Sans normalisation, deux groupes de points distincts semblent visibles sur la projection dans le premier plan factoriel. Avc normalisation, nous n'observons plus cette répartition, ce qui est conforme à une ACP sur des données aléatoires.

## 10.3 Appliquez une ACP sur des variations de cours d'actions

Appliquez une ACP sur le dataset contenu dans le fichier `data/company-stock-movements-2010-2015-incl.csv` qui contient les variations quotidiennes des valeurs de titres boursiers en fin de séance par rapport à la valeur de la veille de 2010 à 2015.

Note : précisez à Pandas lors de la lecture du fichier CSV que la première colonne contient l'index (paramètre `index_col=0`).

Pour visualiser ces variations pour un ou plusieurs titres vous pouvez utiliser le code suivant (une fois votre dataframe chargée) : `df.loc[['Goldman Sachs', 'Amazon']].transpose().plot(figsize=(15, 7))`.

Attention : l'analyse des cercles de corrélations n'est pas utile pour cette exercice.

In [ ]:
df_stock = pd.read_csv('/data/company-stock-movements-2010-2015-incl.csv', index_col=0)
print(df.shape)
df_stock.head()

Vérifions si ce dataset contient des valeurs manquantes :

In [ ]:
pd.isnull(df_stock).any().any()

### 10.3.1 Visualisation des mouvements

Nous pouvons visualiser les mouvements d'un ou plusieurs titres sur un graphique. Le résultat semble peu exploitable pour identifier des titres ayant des variations de valeurs proches.

In [ ]:
df_stock.loc[['Goldman Sachs', 'Amazon']].transpose().plot(figsize=(15, 7))

### 10.3.2 Analyse en Composantes Principales

Nous allons réduire le nombre de dimensions à l'aide d'une ACP pour projeter les titres sur le plan des deux premières composantes principales. Appliquons une ACP en précisant que l'on souhaite obtenir suffisament de composantes pour expliquer 95% de la variance :

In [ ]:
scaler = StandardScaler()
# remplacer par X = df.values pour une ACP sur les données non normalisées
X = scaler.fit_transform(df_stock.values)
X = normalize(df_stock.values)
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X)

In [ ]:
pca.explained_variance_ratio_

Les deux premières composantes principales n'expliquent que 13% de la variance de nos données :

In [ ]:
display_scree_plot(pca)

In [ ]:
pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[1]

Nombre de composantes nécessaires pour atteindre 95% de la variance :

In [ ]:
pca.n_components_

### 10.3.3 Projection sur le premier plan factoriel

Visualisons les titres projetés sur le plan des deux premières composantes principales. Pour cela nous construisons une dataframe contenant le résultat de notre ACP puis nous affichons le graphique correspondant à cette projection :

In [ ]:
df_stock_pca = pd.DataFrame(X_pca, index=df_stock.index)
df_stock_pca

In [ ]:
fig = px.scatter(df_stock_pca, x=0, y=1, hover_name=df_stock.index, text=df_stock.index)
fig.show()

Plusieurs groupes de titres sont identifiables sur cette représentation tel qu'un groupe composé des titres technologiques (Yahoo, Amazon, Google ...) et un autre composé des titres des majors pétrolières (Total, Chevron, Shell ...).

### 10.3.4 Visualisation de quelques titres proches

Pour visualiser les variations de certains titres, nous allons créer une dataframe contenant les données normalisées (nous nous intéressons aux variations relatives et non aux variations absolues).

Mais la visualisation des données brutes de plusieurs titres proches dans notre espace réduit à l'aide d'une ACP ne permet pas d'apprécier leurs comportements similaires :

In [ ]:
df_stock_normalized = pd.DataFrame(X, columns=df_stock.columns, index=df_stock.index)
df_stock_normalized.head()

In [ ]:
df_stock_normalized.loc[['Goldman Sachs', 'Bank of America', 'Microsoft', 'HP']].transpose().plot(figsize=(15, 7))

Mais nous pouvons lisser ces courbes à l'aide de la méthode [pandas.DataFrame.rolling](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html) de pandas. Le résultat est plus satisfaisant et permet d'observer des similarités dans les variations de la valeur des titres (nous présentons ici plusieurs comparaisons pertinentes) :

In [ ]:
df_stock_normalized.loc[['Goldman Sachs', 'Bank of America', 'Microsoft', 'HP']] \
    .transpose().rolling(window=200, center=True).mean().plot(figsize=(15, 7))

In [ ]:
df_stock_normalized.loc[['Goldman Sachs', 'Bank of America', 'Colgate-Palmolive', 'Kimberly-Clark']] \
    .transpose().rolling(window=200, center=True).mean().plot(figsize=(15, 7))

In [ ]:
df_stock_normalized.loc[['Goldman Sachs', 'Bank of America', 'Total', 'Chevron']] \
    .transpose().rolling(window=200, center=True).mean().plot(figsize=(15, 7))

**Bonus : Clustering hierarchique**

Une méthode de clustering devrait permettre d'obtenir des groupes de titres similaires. Appliquons un clustering hiérachique en utilisant la librairie scipy qui permet d'obtenir une visualisation du dendrogramme.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

linked = linkage(df_stock_normalized, 'ward')

labels = df_stock_normalized.index

plt.figure(figsize=(15, 7))
dendrogram(linked,
            orientation='top',
            labels=labels,
            distance_sort='descending',
            show_leaf_counts=True,
            leaf_font_size=10)

plt.show()

Scikit-learn fournit une classe permettant d'appliquer un clustering hiérarchique en précisant le nombre de clusters que nous souhaitons obtenir :

In [ ]:
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=9, affinity='euclidean', linkage='ward')  
labels = cluster.fit_predict(df_stock_normalized)

Finalement, nous pouvons ajouter les labels des clusters pour chaque titre dans notre dataframe d'origine, puis afficher le label pour chaque titre en ordonnant le résultat par le numéro de label :

In [ ]:
df_stock['label'] = labels
df_stock['label'].sort_values()

Nous observons que cette méthode permet d'identifier des clusters de titres comprenant des entreprises dont l'activité est proche. Par exemple, le cluster 4 comprend des entreprises de la défense (Boeing, Lookheed Martin et Northrop Grumman). Mais certains clusters semblent peu cohérents, comme le cluster 1 qui contient aussi bien Apple que McDonalds. Il serait intéressant de tester des variations sur le nombre de clusters.

## 10.4 Compréssion d'image

L'ACP peut aussi être utile pour compresser des données. Nous allons appliquer cette méthode pour compréser une image. La cellule suivante charge les modules nécessaires, charge et affiche l'image de test :

In [ ]:
import matplotlib.image as mpimg
from matplotlib import image

img = image.imread('/data/bird.png')
print(img.shape)
plt.axis('off')
plt.imshow(img)

L'image contient 256 lignes, 349 colonnes et 4 canaux : un par couleur (RGB) et un pour le canal alpha (la transparence, une spécificité du format PNG). Nous commencons par transformer notre image pour obtenir un tableau de 256 lignes et 1396 colonnes (les 4 composantes sont simplement concaténées) :

In [ ]:
print(img.shape)
img_reshaped = np.reshape(img, (img.shape[0], img.shape[1] * img.shape[2]))
img_reshaped.shape

Puis nous appliquons notre ACP en précisant le nombre de composantes que nous souhaitons obtenir, dans notre exemple, notre image compressée contiendra donc 256 lignes et 32 colonnes (nos 32 composantes principales) :

In [ ]:
pca = PCA(32)
img_compressed = pca.fit_transform(img_reshaped)
img_compressed.shape

Nous avons donc un taux de compression d'environ 43 (notre image compressée est 43 fois plus légère que notre image d'origine, attention : il faudrait aussi compter le poids des coefficients de nos composantes principales pour être exact) :

In [ ]:
img.size, img_compressed.size, img.size / img_compressed.size

Nos premières composantes principales expliquent une grande part de la variance de notre image, nos 32 composantes expliquent plus de 98% de sa variance :

In [ ]:
display_scree_plot(pca)

In [ ]:
print(np.sum(pca.explained_variance_ratio_))

Finalement, nous pouvons utiliser la méthode `inverse_transform` de la classe `PCA` de scikit-learn pour retrouver les données projetées dans le même espace dimensionnel que l'image source (i.e. : 1396 colonnes, onc 349 colonnes sur 4 canaux) et afficher cette image :

In [ ]:
image_reconstructed = pca.inverse_transform(img_compressed)
print(image_reconstructed.shape)
# reshaping 1396 back to the original 349 * 4
image_reconstructed = np.reshape(image_reconstructed, (img.shape[0],img.shape[1],img.shape[2]))
print(image_reconstructed.shape)
plt.axis('off')
plt.imshow(image_reconstructed)

La cellule suivante affiche un slider permettant de choisir le nombre de composantes que l'on souhaite conserver et affiche l'image d'origine et l'image compressée côte à côte :

In [ ]:
from ipywidgets import Layout, interact
import ipywidgets as widgets

def compress_image(n_components=32):
    pca = PCA(n_components)
    img_compressed = pca.fit_transform(img_reshaped)
    image_reconstructed = pca.inverse_transform(img_compressed)
    image_reconstructed = np.reshape(image_reconstructed, (img.shape[0],img.shape[1],img.shape[2]))
    f, axarr = plt.subplots(1,2, figsize=(20,10))
    axarr[0].imshow(img)
    axarr[0].axis('off')
    axarr[1].imshow(image_reconstructed)
    plt.axis('off')

interact(
    compress_image,
    n_components=widgets.IntSlider(
        min=1, max=256, step=1, value=32,
        layout=Layout(width='500px'),
        continuous_update = False
    )
);